In [ ]:
import os
import zipfile

# Unzip the dataset
zip_path = '/content/test.zip'
extract_path = 'data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Define the directories
base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')

# Create directories for training data
os.makedirs(os.path.join(train_dir, 'cats'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'dogs'), exist_ok=True)

# Assume that the unzipped images are in the base directory and move them to the respective folders
for file in os.listdir(base_dir):
    if 'cat' in file:
        os.rename(os.path.join(base_dir, file), os.path.join(train_dir, 'cats', file))
    elif 'dog' in file:
        os.rename(os.path.join(base_dir, file), os.path.join(train_dir, 'dogs', file))


Load and Preprocess the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation')


Found 112 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


 Build the Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Train the Model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size), # Ensure steps_per_epoch is at least 1
    validation_data=validation_generator,
    validation_steps=max(1, validation_generator.samples // validation_generator.batch_size), # Ensure validation_steps is at least 1
    epochs=8)

Epoch 1/8
3/3 [==============================] - 8s 2s/step - loss: 0.8862 - accuracy: 0.4875 - val_loss: 0.7069 - val_accuracy: 0.5000
Epoch 2/8
3/3 [==============================] - 7s 2s/step - loss: 0.6976 - accuracy: 0.5208 - val_loss: 0.7005 - val_accuracy: 0.5000
Epoch 3/8
3/3 [==============================] - 5s 2s/step - loss: 0.7012 - accuracy: 0.4750 - val_loss: 0.7015 - val_accuracy: 0.5000
Epoch 4/8
3/3 [==============================] - 6s 2s/step - loss: 0.6979 - accuracy: 0.5104 - val_loss: 0.6949 - val_accuracy: 0.5000
Epoch 5/8
3/3 [==============================] - 7s 3s/step - loss: 0.6948 - accuracy: 0.4750 - val_loss: 0.6908 - val_accuracy: 0.6429
Epoch 6/8
3/3 [==============================] - 6s 2s/step - loss: 0.6552 - accuracy: 0.7812 - val_loss: 0.6949 - val_accuracy: 0.4643
Epoch 7/8
3/3 [==============================] - 7s 2s/step - loss: 0.6269 - accuracy: 0.6750 - val_loss: 0.7035 - val_accuracy: 0.5000
Epoch 8/8
3/3 [==============================] -

 Evaluate the Model

In [ ]:
# Evaluate the model on validation data
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {validation_loss}')
print(f'Validation Accuracy: {validation_accuracy}')


1/1 [==============================] - 1s 768ms/step - loss: 0.7015 - accuracy: 0.6071
Validation Loss: 0.701529860496521
Validation Accuracy: 0.6071428656578064


Make Predictions

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(model, img_path):
    # Use a valid image path from your dataset
    img_path = '/content/data/train/dogs/dogs/dog_213.jpg'  # Replace with a real image path
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    return 'dog' if prediction[0] > 0.5 else 'cat'

# Example usage
print(predict_image(model, img_path))


1/1 [==============================] - 0s 47ms/step
dog
